In [1]:
## Create network model for the Balearic Islands --- adding solar

In [2]:
%%capture
# install pandapower
!pip install pandapower

In [3]:
# imports
import pandas as pd
import pandapower as pp
from pandapower.plotting.plotly import simple_plotly

In [4]:
# path to folders with data
drive_path = './'

In [5]:
'''The board is committed to at least 80% of the daily (24 hour) summer energy
to be supplied by solar PV on the Islands. Treat this exercise as if you are 
a consultant to Red Électricia de España (REE). The results are to be presented
to the REE Board.'''

# summer scenario  --- we are running this scenario below
  # date = 9 Aug 2019 @ 13:31
  # max demand = 1303 MW

# need 3500 MW solar to supply 81% of energy in the highest demand summer day
  # max net load @ 1 PM
    # capacity factor = 74.8%
  # scenario 1: ceteris paribus, add solar, curtail 30%
  # scenario 2: add trafo at bus 1, add line, add solar
  # scenario 3: add baterries, add solar

'The board is committed to at least 80% of the daily (24 hour) summer energy\nto be supplied by solar PV on the Islands. Treat this exercise as if you are \na consultant to Red Électricia de España (REE). The results are to be presented\nto the REE Board.'

In [6]:
# NOTE: we are using the biggest lines available (Condor), run code, then move to smaller lines if loading percent is too small

In [7]:
# create network
net = pp.create_empty_network() 

In [8]:
# read bus data -- columns: vn_kv, name, geodata
bus_df = pd.read_csv(f'{drive_path}balearic_data_clean - buses.csv', encoding='latin-1')

In [9]:
# create 18 buses
b = {}

for i in bus_df['bus_id']:
  b[i] = pp.create_bus(net, vn_kv = bus_df.loc[bus_df['bus_id']==i, 'vn_kv'].values[0], name = bus_df.loc[bus_df['bus_id']==i, 'name'].values[0], geodata = eval(bus_df.loc[bus_df['bus_id']==i, 'geodata'].values[0]))

In [10]:
# create one grid connection point
connect_bus = b[1]
vm_pu = 1

pp.create_ext_grid(net, bus=connect_bus, vm_pu=vm_pu, name = "Slack bus")

0

In [11]:
# create trafos

# bus 1 Santa Ponca (132) / bus 2 Santa Ponca (220)
hv_bus = net.bus[net.bus.name == 'Santa Ponca (220)'].index.values[0]
lv_bus = net.bus[net.bus.name == 'Santa Ponca (132)'].index.values[0]
# data from https://tinyurl.com/2p9xceax
sn_mva = 320
vn_hv_kv = 220
vn_lv_kv = 132
# used parameters from 100 MVA 220/110 kV standard type
vk_percent = 12
vkr_percent = 0.26
pfe_kw = 55
i0_percent = 0.06 
pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

# bus 5	Portol (Cas Tresorer) (220) / bus 5.1	Portol (Cas Tresorer) (66)
hv_bus = net.bus[net.bus.name == 'Portol (Cas Tresorer) (220)'].index.values[0]
lv_bus = net.bus[net.bus.name == 'Portol (Cas Tresorer) (66)'].index.values[0]
# data from https://www.ree.es/sites/default/files/01_ACTIVIDADES/Documentos/Mapas-de-red/mapa_transporte_iberico_2018.pdf
sn_mva = 480
vn_hv_kv = 220
vn_lv_kv = 66
# used parameters from 100 MVA 220/110 kV standard type
vk_percent = 12
vkr_percent = 0.26
pfe_kw = 55
i0_percent = 0.06
pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)


# 10	Es Bessons (220) / 11	Es Bessons (132)
hv_bus = net.bus[net.bus.name == 'Es Bessons (220)'].index.values[0]
lv_bus = net.bus[net.bus.name == 'Es Bessons (132)'].index.values[0]
# data from https://tinyurl.com/56sh33a9
sn_mva = 446
vn_hv_kv = 220
vn_lv_kv = 132
# used parameters from 100 MVA 220/110 kV standard type
vk_percent = 12
vkr_percent = 0.26
pfe_kw = 55
i0_percent = 0.06
#pp.create_transformer(net, hv_bus=hv_bus, lv_bus=lv_bus, std_type='100 MVA 220/110 kV')
pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

2

In [12]:
# read df for lines -- from_bus, to_bus, length_km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka
line_df = pd.read_csv(f'{drive_path}balearic_data_clean - lines.csv', encoding='latin-1')

In [13]:
# create lines
for i in line_df.index:
  pp.create_line_from_parameters(net, 
                                 from_bus=b[line_df.loc[i, 'from_bus']],
                                 to_bus=b[line_df.loc[i, 'to_bus']],
                                 length_km=line_df.loc[i, 'length_km'],
                                 r_ohm_per_km=line_df.loc[i, 'r_ohm_per_km'],
                                 x_ohm_per_km=line_df.loc[i, 'x_ohm_per_km'],
                                 c_nf_per_km=line_df.loc[i, 'c_nf_per_km'],
                                 max_i_ka=line_df.loc[i, 'max_i_ka']
                                 ) 

In [14]:
# read df for loads -- geolocation, bus (nearest from geolocation), p_mw
load_df = pd.read_csv(f'{drive_path}balearic_data_clean - loads.csv', encoding='latin-1').dropna()

In [15]:
# create loads
p_mw = 'summer_p_mw'
q_mvar = 'summer_q_mvar'

for i in load_df['bus_id']:
  pp.create_load(net, 
                 b[i], 
                 p_mw=load_df.loc[load_df['bus_id']==i, p_mw].values[0], 
                 q_mvar=load_df.loc[load_df['bus_id']==i, q_mvar].values[0])

In [16]:
# look at share of loads by bus

load_share = load_df[['bus_id', p_mw]]
load_share['share'] = load_share[p_mw].apply(lambda x: x / sum(load_share[p_mw]))
load_dict = dict(zip(load_share['bus_id'], load_share['share']*4000))

In [17]:
# choose scenario to run
sc = 'sc1'

In [18]:
# create solar gen 
# scenario 1: 
  # relies heavily on bus 0 (may impact tourism) and 1 for generation
  # limiting factor is trafo 0
  # need to curtail at least 30% of solar power


if sc == 'sc1':

  total_solar = 3500
  max_solar_cf = 0.748
  curtail = 0.25

  gen_mw = {0.0: 500,
            1.0: 650,
            2.0: 150,
            3.0: 150,
            4.0: 150,
            5.0: 150,
            5.1: 100,
            6.0: 200,
            7.0: 200,
            8.0: 200,
            9.0: 200,
            10.0: 300,
            11.0: 300,
            12.0: 50,
            13.0: 50,
            14.0: 50,
            15.0: 50,
            16.0: 50}

  for i in gen_mw.keys():
    pp.create_gen(net, 
                  b[i], 
                  p_mw = gen_mw[i]*max_solar_cf*(1-curtail), 
                  vm_pu=vm_pu)

In [19]:
# create solar gen 
# scenario 2: 
  # add 320MVA for trafo 0
  # add line from bus 2 to bus 3
  # no curtailement
  # max loading: line = 94.3%, trafo = 92.4%
  # export power = 1528 MW
  # relies heavily on bus 1 for generation

if sc == 'sc2':

  # add trafo for bus 1 Santa Ponca (132) / bus 2 Santa Ponca (220)
  hv_bus = net.bus[net.bus.name == 'Santa Ponca (220)'].index.values[0]
  lv_bus = net.bus[net.bus.name == 'Santa Ponca (132)'].index.values[0]
  # data from https://tinyurl.com/2p9xceax
  sn_mva = 320*1.5
  vn_hv_kv = 220
  vn_lv_kv = 132
  # used parameters from 100 MVA 220/110 kV standard type
  vk_percent = 12
  vkr_percent = 0.26
  pfe_kw = 55
  i0_percent = 0.06 
  pp.create_transformer_from_parameters(net, hv_bus=hv_bus, lv_bus=lv_bus, sn_mva=sn_mva, vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv, vkr_percent=vkr_percent, vk_percent=vk_percent, pfe_kw=pfe_kw, i0_percent=i0_percent)

  # add line for scenario 2
  for i in [1]:
    pp.create_line_from_parameters(net, 
                                  from_bus=b[line_df.loc[i, 'from_bus']],
                                  to_bus=b[line_df.loc[i, 'to_bus']],
                                  length_km=line_df.loc[i, 'length_km'],
                                  r_ohm_per_km=line_df.loc[i, 'r_ohm_per_km'],
                                  x_ohm_per_km=line_df.loc[i, 'x_ohm_per_km'],
                                  c_nf_per_km=line_df.loc[i, 'c_nf_per_km'],
                                  max_i_ka=line_df.loc[i, 'max_i_ka']
                                  ) 

  total_solar = 3500
  max_solar_cf = 0.748
  curtail = 0.0

  gen_mw = {0.0: 350,
            1.0: 750,
            2.0: 200,
            3.0: 200,
            4.0: 200,
            5.0: 200,
            5.1: 150,
            6.0: 200,
            7.0: 200,
            8.0: 200,
            9.0: 200,
            10.0: 200,
            11.0: 200,
            12.0: 50,
            13.0: 50,
            14.0: 50,
            15.0: 50,
            16.0: 50}

  for i in gen_mw.keys():
    pp.create_gen(net, 
                  b[i], 
                  p_mw = gen_mw[i]*max_solar_cf*(1-curtail), 
                  vm_pu=vm_pu,   # defined above
                  )


In [20]:
# create solar gen 
# scenario 3:
  # add batteries
  # relies heavily on bus 1 for generation
  # limiting factor is trafo 0

if sc == 'sc3':

  # add storage
  duration = 4
  s_mw = {0.0: 100,
          1.0: 0,
          2.0: 0,
          3.0: 100,
          4.0: 100,
          5.0: 0,
          5.1: 0,
          6.0: 0,
          7.0: 100,
          8.0: 100,
          9.0: 0,
          10.0: 0,
          11.0: 20,
          12.0: 0,
          13.0: 0,
          14.0: 0,
          15.0: 0,
          16.0: 0}
  
  for i in s_mw.keys():
    pp.create_storage(net, 
                      b[i], 
                      p_mw=s_mw[i], 
                      max_e_mwh=s_mw[i]*duration, 
                      )

  # add solar
  total_solar = 3500
  max_solar_cf = 0.748
  curtail = 0

  gen_mw = {0.0: 500,
            1.0: 650,
            2.0: 150,
            3.0: 150,
            4.0: 150,
            5.0: 150,
            5.1: 100,
            6.0: 200,
            7.0: 200,
            8.0: 200,
            9.0: 200,
            10.0: 300,
            11.0: 300,
            12.0: 50,
            13.0: 50,
            14.0: 50,
            15.0: 50,
            16.0: 50}

  for i in gen_mw.keys():
    pp.create_gen(net, 
                  b[i], 
                  p_mw = gen_mw[i]*max_solar_cf*(1-curtail), 
                  vm_pu=vm_pu)

In [21]:
pp.runpp(net, algorithm='nr');

In [22]:
simple_plotly(net)

In [23]:
print(max(net.res_trafo.loading_percent))

94.42342523626606


In [24]:
print(max(net.res_line.loading_percent))

90.26635590083374


In [25]:
print(net.res_ext_grid.p_mw)

0   -652.987279
Name: p_mw, dtype: float64


In [26]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,105.070409,-40.135910,-100.221852,51.051567,4.848557,10.915657,0.491952,0.491952,0.491952,1.0,6.087250,1.0,0.000000,90.266356
1,-217.816542,9.118578,217.883287,-7.353064,0.066745,1.765514,0.572121,0.572121,0.572121,1.0,6.495006,1.0,6.959344,58.679062
2,-122.868297,10.051815,122.982832,-8.537001,0.114535,1.514814,0.323523,0.323523,0.323523,1.0,6.959344,1.0,7.665392,33.181835
3,-123.919616,10.137823,124.035131,-8.610047,0.115515,1.527776,0.326291,0.326291,0.326291,1.0,6.959344,1.0,7.665392,33.465755
4,-139.891309,11.195942,139.984345,-9.965469,0.093036,1.230473,0.368293,0.368293,0.368293,1.0,7.665392,1.0,8.169191,37.773678
5,-136.031280,12.390970,136.347885,-8.203632,0.316605,4.187339,0.358467,0.358467,0.358467,1.0,7.665392,1.0,9.426892,36.765895
6,-27.121159,2.061387,27.122776,-2.040000,0.001617,0.021387,0.071380,0.071380,0.071380,1.0,8.169191,1.0,8.214371,7.321013
7,-113.717812,9.854549,113.906715,-7.356152,0.188904,2.498397,0.299550,0.299550,0.299550,1.0,8.169191,1.0,9.426892,30.723109
8,0.418118,-0.031604,-0.418116,0.031624,0.000001,0.000020,0.001100,0.001100,0.001100,1.0,9.426892,1.0,9.424212,0.112862
9,0.436511,-0.032994,-0.436509,0.033015,0.000002,0.000020,0.001149,0.001149,0.001149,1.0,9.426892,1.0,9.424212,0.117827
